## Creating horses.db

**NOTES:** 
- Only include race distances: 1200  1400  1650  1000  1600  1800  

In [1]:
import pandas as pd
import numpy as np
import sqlite3
conn = sqlite3.connect('horses.db')
curs = conn.cursor()

In [2]:
runs = pd.read_csv('data/runs.csv')
races = pd.read_csv('data/races.csv')
runs.columns

Index(['race_id', 'horse_no', 'horse_id', 'result', 'won', 'lengths_behind',
       'horse_age', 'horse_country', 'horse_type', 'horse_rating',
       'horse_gear', 'declared_weight', 'actual_weight', 'draw',
       'position_sec1', 'position_sec2', 'position_sec3', 'position_sec4',
       'position_sec5', 'position_sec6', 'behind_sec1', 'behind_sec2',
       'behind_sec3', 'behind_sec4', 'behind_sec5', 'behind_sec6', 'time1',
       'time2', 'time3', 'time4', 'time5', 'time6', 'finish_time', 'win_odds',
       'place_odds', 'trainer_id', 'jockey_id'],
      dtype='object')

**Filter necessary columns:**

In [3]:
runs_reduced = runs[['race_id', 'horse_no', 'horse_id', 'time1','time2', 'time3', 'time4', 'time5', 'time6', 'finish_time']]
races_reduced = races[['race_id', 'surface', 'distance', 'going']]

**Drop any outliers:**

In [4]:
#sorted(runs_reduced['time3'], reverse=True)
#sorted(runs_reduced['time1'])

In [5]:
runs_reduced = runs_reduced[runs_reduced['time3'] < 50]
runs_reduced = runs_reduced[runs_reduced['time2'] < 50]
runs_reduced = runs_reduced[runs_reduced['time1'] < 50]

---

**Create database:**
- Only two tables needed- one for races, one for horses

In [6]:
curs.execute("DROP TABLE IF EXISTS tRuns;")
curs.execute("DROP TABLE IF EXISTS tRaces;")

runs_reduced.to_sql('tRuns', 
            conn,
            index=False)
races_reduced.to_sql('tRaces', 
            conn,
            index=False)

6349

**tRuns:**

In [7]:
sql_tRuns = "SELECT * FROM tRuns;"
pd.read_sql(sql_tRuns, conn)

,race_id,horse_no,horse_id,time1,time2,time3,time4,time5,time6,finish_time
0,0,1,3917,13.85,21.59,23.86,24.62,NaN,NaN,83.92
1,0,2,2157,14.57,21.99,23.30,23.70,NaN,NaN,83.56
2,0,3,858,13.69,21.59,23.90,24.22,NaN,NaN,83.40
3,0,4,1853,14.09,21.83,23.70,24.00,NaN,NaN,83.62
4,0,5,2796,14.77,21.75,23.22,23.50,NaN,NaN,83.24
...,...,...,...,...,...,...,...,...,...,...
79438,6348,10,1238,26.14,23.98,23.53,22.52,NaN,NaN,96.17
79439,6348,11,985,25.82,24.10,23.77,22.61,NaN,NaN,96.30
79440,6348,12,3147,26.58,24.06,23.37,22.43,NaN,NaN,96.44
79441,6348,13,1982,26.58,24.02,23.45,22.29,NaN,NaN,96.34


**tRaces:**

In [8]:
sql_tRaces = "SELECT * FROM tRaces;"
pd.read_sql(sql_tRaces, conn)

,race_id,surface,distance,going
0,0,0,1400,GOOD TO FIRM
1,1,0,1200,GOOD TO FIRM
2,2,0,1400,GOOD TO FIRM
3,3,0,1200,GOOD TO FIRM
4,4,0,1600,GOOD TO FIRM
...,...,...,...,...
6344,6344,0,1400,GOOD
6345,6345,0,1600,GOOD
6346,6346,0,2000,GOOD
6347,6347,0,1200,GOOD


---
**Queries for obtaining top speed, endurance, and consistency data:**

(Example using 1600m race)

In [9]:
### Query for top speed:
### Average of top three times at given distance for each horse
sql = """
WITH ranked_times AS 
(
  SELECT horse_id, finish_time, ROW_NUMBER() OVER (PARTITION BY horse_id ORDER BY finish_time) AS row_num
  FROM tRuns 
  JOIN tRaces USING(race_id)
  WHERE distance = 1600
)
SELECT horse_id, num_races, avg(finish_time) as top_speed
FROM (SELECT horse_id, count(horse_id) as num_races FROM ranked_times GROUP BY horse_id)
JOIN ranked_times USING(horse_id)
WHERE row_num <= 3
    AND num_races >= 10
GROUP BY horse_id;
"""

### Query for ungrouped time data:
### Used for calculating consistency and endurance
sql2 = """
WITH horses_dist AS 
(
  SELECT horse_id, time1, time2, time3, time4, time5, time6, finish_time
  FROM tRuns 
  JOIN tRaces USING(race_id)
  WHERE distance = 1600
)
SELECT horse_id, time1, time2, time3, time4, time5, time6, finish_time
FROM (SELECT horse_id, count(horse_id) as num_races FROM horses_dist GROUP BY horse_id)
JOIN horses_dist USING(horse_id)
WHERE num_races >= 10
ORDER BY horse_id;
"""

In [10]:
grouped = pd.read_sql(sql, conn)
grouped

,horse_id,num_races,top_speed
0,0,25,94.196667
1,25,10,95.773333
2,52,10,95.706667
3,53,11,95.340000
4,70,10,95.030000
...,...,...,...
187,4359,11,95.760000
188,4376,10,96.126667
189,4384,10,96.216667
190,4386,11,94.653333


**Top speed:**

In [11]:
d = pd.read_sql(sql,conn)
d['rating'] = -pd.qcut(d['top_speed'], 8, labels=False) + 8
samples = d[d.rating == 7]
xbar, sigma = samples.top_speed.mean(), samples.top_speed.std()
print(xbar, sigma)
d

94.46388888888889 0.21202877087305735


,horse_id,num_races,top_speed,rating
0,0,25,94.196667,7
1,25,10,95.773333,3
2,52,10,95.706667,3
3,53,11,95.340000,5
4,70,10,95.030000,6
...,...,...,...,...
187,4359,11,95.760000,3
188,4376,10,96.126667,2
189,4384,10,96.216667,1
190,4386,11,94.653333,7


In [12]:
ungrouped = pd.read_sql(sql2,conn)
ungrouped

,horse_id,time1,time2,time3,time4,time5,time6,finish_time
0,0,24.84,24.42,23.87,23.61,None,None,96.74
1,0,24.91,22.65,23.39,23.44,None,None,94.39
2,0,24.92,23.74,24.33,23.88,None,None,96.87
3,0,24.94,23.57,23.91,23.96,None,None,96.38
4,0,25.05,23.21,23.52,23.38,None,None,95.16
...,...,...,...,...,...,...,...,...
2669,4404,25.68,23.52,23.60,23.85,None,None,96.65
2670,4404,25.70,23.82,24.58,22.88,None,None,96.98
2671,4404,25.80,23.01,23.42,23.25,None,None,95.48
2672,4404,25.86,23.29,24.41,23.24,None,None,96.80


**Endurance:**

In [13]:
s = pd.read_sql(sql2, conn)
s['time_diff'] = s.time4 - s.time2
s = s[['horse_id','time_diff']]
s = s.groupby(['horse_id'],as_index=False).mean()

s['rating'] = -pd.qcut(s['time_diff'], 8, labels=False) + 8
samples = s[s.rating == 7]

xbar, sigma = samples.time_diff.mean(), samples.time_diff.std()
print(xbar, sigma)
s

-0.4367352021956803 0.06240233423823019


,horse_id,time_diff,rating
0,0,-0.251600,6
1,25,1.792000,1
2,52,-0.190000,6
3,53,-0.508182,7
4,70,0.569000,2
...,...,...,...
187,4359,0.277273,3
188,4376,0.466000,3
189,4384,-0.206000,6
190,4386,-0.200000,6


**Consistency:**

In [14]:
s = pd.read_sql(sql2, conn)
times = s[['horse_id','finish_time']].copy()
st_devs = times.groupby(['horse_id'],as_index=False).std()
st_devs['rating'] = -pd.qcut(st_devs['finish_time'], 8, labels=False) + 8
st_devs

,horse_id,finish_time,rating
0,0,0.875114,4
1,25,0.832613,4
2,52,0.692227,6
3,53,0.999453,3
4,70,1.135890,2
...,...,...,...
187,4359,0.675764,6
188,4376,0.377595,8
189,4384,0.590409,7
190,4386,0.729594,5
